# Requirements
* GPU: Minimum of a T4 GPU.
* RAM: High-RAM compute instance required.

# Installation
Expect an long setup time due to the need for compiling c extensions for dependencies and installing CUDA 11.7.

# Performance Notes
* Performance: Using higher-end GPUs like the V100 offers large performance gains, but still requires the same sunk-cost of compilation and setup. If you are generating lots of assets, you could see benefit of using a higher end gpu.
* Recommendation: Stick with the minimum GPU requirement (T4 or better) to balance between performance and cost.

# Benchmarking

Leave One Out

| GPU      | Speed (it/s) |
|----------|--------------|
| RTX 3070 | 73.35        |
| T4       | 64.78        |
| V100     | 109.35       |

Coarse 3DGS Generation

| GPU      | Speed (it/s) |
|----------|--------------|
| RTX 3070 | 73.04        |
| T4       | 58.89        |
| V100     | 95.44        |

LORA Training

| GPU      | Speed (it/s) |
|----------|--------------|
| RTX 3070 | 0.05         |
| T4       | 29.45        |
| V100     | 142.35       |

Gaussian Repair

| GPU      | Speed (it/s) |
|----------|--------------|
| RTX 3070 | N/A          |
| T4       | 1.27         |
| V100     | 3.49         |

# Check Hardware Requirements

In [1]:
import os

if 20e9 > (os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')):
    raise Exception("This script requires a High-RAM Instance")

nvcc_cmd = !which nvcc
if not nvcc_cmd:
    raise Exception('This script requires a GPU Instance')

!nvidia-smi

AttributeError: module 'os' has no attribute 'sysconf'

# Install CUDA 11.7

Because the pytorch binary was installed using `cuda-11.7`, we must replace the current cuda with 11.7

In [5]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
!sudo mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.7.0/local_installers/cuda-repo-ubuntu2204-11-7-local_11.7.0-515.43.04-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu2204-11-7-local_11.7.0-515.43.04-1_amd64.deb
!sudo cp /var/cuda-repo-ubuntu2204-11-7-local/cuda-*-keyring.gpg /usr/share/keyrings/
!sudo apt-get -y update
!sudo DEBIAN_FRONTEND=noninteractive apt-get -y install cuda-11-7

'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


'sudo' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'sudo' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'sudo' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'sudo' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'sudo' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


# Set Environment Variables to Point to Installed CUDA

In [6]:
import os

os.environ['CUDA_HOME'] = '/usr/local/cuda-11.7'
if '/usr/local/cuda-11.7/bin' not in os.environ['PATH'].split(':'):
    os.environ['PATH'] = ':'.join(['/usr/local/cuda-11.7/bin', os.environ['PATH']])

# Install GCC-9

We need to downgrade `gcc` and `g++` for it to be compatible

In [7]:
!sudo apt-get install -y software-properties-common
!sudo add-apt-repository -y ppa:ubuntu-toolchain-r/test
!sudo apt-get -y update
!sudo apt-get install -y gcc-9 g++-9

!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 60 --slave /usr/bin/g++ g++ /usr/bin/g++-9
!sudo update-alternatives --config gcc

'sudo' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'sudo' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'sudo' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'sudo' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'sudo' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'sudo' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


# Clone GaussianObject Repository

In [9]:
!git clone https://github.com/GaussianObject/GaussianObject.git --recursive

%cd ./GaussianObject

d:\Repos\6476-team55\GaussianObject


fatal: destination path 'GaussianObject' already exists and is not an empty directory.


In [10]:
!pip install -r requirements.txt

# Build Submodules (NOTE, can take a long time to compile `pytorch3d`)

In [12]:
!pip install -e submodules/diff-gaussian-rasterization
!pip install -e submodules/simple-knn
!pip install -e submodules/pytorch3d
!pip install -e submodules/minLoRA
!pip install -e submodules/CLIP

Obtaining file:///D:/Repos/6476-team55/GaussianObject/submodules/diff-gaussian-rasterization
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "D:\Repos\6476-team55\GaussianObject\submodules\diff-gaussian-rasterization\setup.py", line 13, in <module>
          from torch.utils.cpp_extension import CUDAExtension, BuildExtension
        File "C:\Users\12063\anaconda3\lib\site-packages\torch\__init__.py", line 122, in <module>
          raise err
      OSError: [WinError 127] 找不到指定的程序。 Error loading "C:\Users\12063\anaconda3\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an 

Obtaining file:///D:/Repos/6476-team55/GaussianObject/submodules/simple-knn
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "D:\Repos\6476-team55\GaussianObject\submodules\simple-knn\setup.py", line 13, in <module>
          from torch.utils.cpp_extension import CUDAExtension, BuildExtension
        File "C:\Users\12063\anaconda3\lib\site-packages\torch\__init__.py", line 122, in <module>
          raise err
      OSError: [WinError 127] 找不到指定的程序。 Error loading "C:\Users\12063\anaconda3\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the pa

Obtaining file:///D:/Repos/6476-team55/GaussianObject/submodules/pytorch3d
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "D:\Repos\6476-team55\GaussianObject\submodules\pytorch3d\setup.py", line 15, in <module>
          import torch
        File "C:\Users\12063\anaconda3\lib\site-packages\torch\__init__.py", line 122, in <module>
          raise err
      OSError: [WinError 127] 找不到指定的程序。 Error loading "C:\Users\12063\anaconda3\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for deta

Obtaining file:///D:/Repos/6476-team55/GaussianObject/submodules/minLoRA
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py develop for minLoRA


Obtaining file:///D:/Repos/6476-team55/GaussianObject/submodules/CLIP
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py develop for clip


In [ ]:
!python download_hf_models.py

v1-5-pruned.ckpt: 100% 7.70G/7.70G [00:30<00:00, 250MB/s] 
control_v11f1e_sd15_tile.pth: 100% 1.45G/1.45G [00:04<00:00, 348MB/s]


# Download MIP360 Dataset

Warning Download May Fail

If `gdown` fails to download the dataset, uncomment the `LOAD MIP360 Dataset Through Personal Google Drive` code below and follow the instructions

In [ ]:
%%bash

if gdown 1zZzt7_r3Ie3lWIQDyyB0_4ZfvReL2I7r; then
    mkdir data
    mv mip360.zip data/
    unzip data/mip360.zip -d data/
    rm data/mip360.zip
fi

Archive:  data/mip360.zip
   creating: data/mip360/
   creating: data/mip360/bonsai/
   creating: data/mip360/bonsai/images/
  inflating: data/mip360/bonsai/images/DSCF5627.JPG  
  inflating: data/mip360/bonsai/images/DSCF5797.JPG  
  inflating: data/mip360/bonsai/images/DSCF5625.JPG  
  inflating: data/mip360/bonsai/images/DSCF5719.JPG  
  inflating: data/mip360/bonsai/images/DSCF5669.JPG  
  inflating: data/mip360/bonsai/images/DSCF5834.JPG  
  inflating: data/mip360/bonsai/images/DSCF5732.JPG  
  inflating: data/mip360/bonsai/images/DSCF5594.JPG  
  inflating: data/mip360/bonsai/images/DSCF5691.JPG  
  inflating: data/mip360/bonsai/images/DSCF5663.JPG  
  inflating: data/mip360/bonsai/images/DSCF5708.JPG  
  inflating: data/mip360/bonsai/images/DSCF5760.JPG  
  inflating: data/mip360/bonsai/images/DSCF5621.JPG  
  inflating: data/mip360/bonsai/images/DSCF5841.JPG  
  inflating: data/mip360/bonsai/images/DSCF5578.JPG  
  inflating: data/mip360/bonsai/images/DSCF5747.JPG  
  inflating

# Load MIP360 Dataset Through Personal Google Drive

If the above method doesn't work, you can go to the link below, right click `mip360.zip` file and copy it to your personal google drive. Mount your personal google drive, then load it from there. I placed the `mip360.zip` under a folder called `tmp`, change the path to point to your own google drive location.

https://drive.google.com/drive/folders/1Q5duwHO3jECWK_khLnPM088BxUa3zhko

In [ ]:
# from google.colab import drive

# drive.mount('/gdrive')

# !mkdir data
# !rsync -av /gdrive/MyDrive/tmp/mip360.zip data # changeme to point to your own google drive location where you saved mip360.zip
# !unzip data/mip360.zip -d data/
# !rm data/mip360.zip

# Train Model

## Visual Hull

In [ ]:
!python visual_hull.py \
    --sparse_id 4 \
    --data_dir data/mip360/kitchen \
    --reso 2 --not_vis

Using resized images in data/mip360/kitchen/images_2...
Reading camera 279/279
Generating ellipse path from 279 camera infos ...
theta[0] 0.0
Converting point3d.bin to .ply, will happen only the first time you open the scene.
the sparse id is 4, with 4 frames
the camera center is: tensor([-0.0710, -0.0025,  0.0700], device='cuda:0')
  0% 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100% 200/200 [00:00<00:00, 703.50it/s]
visual hull is Okay, with 22537 points
100% 64/64 [00:00<00:00, 989.56it/s]
visual hull is Okay, with 8616 points


The visual hull is saved in `data/mip360/kitchen/visual_hull_4.ply`.

## Coarse 3DGS

In [ ]:
!python train_gs.py -s data/mip360/kitchen \
    -m output/gs_init/kitchen \
    -r 4 --sparse_view_num 4 --sh_degree 2 \
    --init_pcd_name visual_hull_4 \
    --white_background --random_background

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2024-03-03 22:37:35.232743: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 22:37:35.232795: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 22:37:35.234077: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-03 22:37:36.695075: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find

You can render the coarse model it with

In [ ]:
# render the test set
!python render.py \
    -m output/gs_init/kitchen \
    --sparse_view_num 4 --sh_degree 2 \
    --init_pcd_name visual_hull_4 \
    --white_background --skip_all --skip_train

# render the path
!python render.py \
    -m output/gs_init/kitchen \
    --sparse_view_num 4 --sh_degree 2 \
    --init_pcd_name visual_hull_4 \
    --white_background --render_path

Looking for config file in output/gs_init/kitchen/cfg_args
Config file found: output/gs_init/kitchen/cfg_args
Rendering output/gs_init/kitchen
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100% 528M/528M [00:01<00:00, 334MB/s]
Load

The rendering results are saved in `output/gs_init/kitchen/test/ours_10000` and `output/gs_init/kitchen/render/ours_10000`.

In [ ]:
from IPython.display import Video

Video("/content/GaussianObject/output/gs_init/kitchen/render/ours_10000/renders.mp4", embed=True)

## Leave One Out

In [ ]:
!python leave_one_out_stage1.py -s data/mip360/kitchen \
    -m output/gs_init/kitchen_loo \
    -r 4 --sparse_view_num 4 --sh_degree 2 \
    --init_pcd_name visual_hull_4 \
    --white_background --random_background

!python leave_one_out_stage2.py -s data/mip360/kitchen \
    -m output/gs_init/kitchen_loo \
    -r 4 --sparse_view_num 4 --sh_degree 2 \
    --init_pcd_name visual_hull_4 \
    --white_background --random_background

2024-03-03 22:42:10.631479: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 22:42:10.631530: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 22:42:10.632799: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-03 22:42:11.816704: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Optimizing output/gs_init/kitchen_loo
Output folder: output/gs_init/kitchen_loo/leave_14 [03/03 22:42:12]
Using resized images in /content/GaussianObject/data/mip360/kitchen/images_4... [03/03 22:42:14]
Reading camera 279/279 [03/03 22:42:19

## LoRA Fine-Tuning

In [ ]:
!python train_lora.py --exp_name controlnet_finetune/kitchen \
    --prompt xxy5syt00 --sh_degree 2 --resolution 4 --sparse_num 4 \
    --data_dir data/mip360/kitchen \
    --gs_dir output/gs_init/kitchen \
    --loo_dir output/gs_init/kitchen_loo \
    --bg_white --sd_locked --train_lora --use_prompt_list \
    --add_diffusion_lora --add_control_lora --add_clip_lora

No module 'xformers'. Proceeding without it.
ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
tokenizer_config.json: 100% 905/905 [00:00<00:00, 4.74MB/s]
vocab.json: 100% 961k/961k [00:00<00:00, 2.96MB/s]
merges.txt: 100% 525k/525k [00:00<00:00, 2.14MB/s]
special_tokens_map.json: 100% 389/389 [00:00<00:00, 2.02MB/s]
tokenizer.json: 100% 2.22M/2.22M [00:00<00:00, 12.9MB/s]
config.json: 100% 4.52k/4.52k [00:00<00:00, 21.6MB/s]
model.safetensors: 100% 1.71G/1.71G [00:04<00:00, 369MB/s]
Loaded model config from [./models/control_v11f1e_sd15_tile.yaml]
Loaded state_dict from [./models/v1-5-pruned.ckpt]
Loaded state_dict from [./models/control_v11f1e_sd15_tile.pth]
Using resized images in data/mip360/kitchen/images_4...
Load 25 images for leave 14
Load 25 images for leave 70
Load 25 images for

## Gaussian Repair

In [ ]:
!python train_repair.py \
    --config configs/gaussian-object.yaml \
    --train --gpu 0 \
    tag="kitchen" \
    system.init_dreamer="output/gs_init/kitchen" \
    system.exp_name="output/controlnet_finetune/kitchen" \
    system.refresh_size=8 \
    data.data_dir="data/mip360/kitchen" \
    data.resolution=4 \
    data.sparse_num=4 \
    data.prompt="a photo of a xxy5syt00" \
    data.refresh_size=8 \
    system.sh_degree=2

Seed set to 0
[INFO] Using resized images in data/mip360/kitchen/images_4...
[INFO] init ply file from iter 10000
100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 147MiB/s]
[INFO] GPU available: True (cuda), used: True
[INFO] TPU available: False, using: 0 TPU cores
[INFO] IPU available: False, using: 0 IPUs
[INFO] HPU available: False, using: 0 HPUs
2024-03-03 23:57:10.145339: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 23:57:10.145390: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 23:57:10.146807: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been r

The final 3DGS representation is saved in `output/gaussian_object/kitchen/save/last.ply`. You can render it with

In [ ]:
# render the test set
!python render.py \
    -m output/gs_init/kitchen \
    --sparse_view_num 4 --sh_degree 2 \
    --init_pcd_name visual_hull_4 \
    --white_background --skip_all --skip_train \
    --load_ply output/gaussian_object/kitchen/save/last.ply

# render the path
!python render.py \
    -m output/gs_init/kitchen \
    --sparse_view_num 4 --sh_degree 2 \
    --init_pcd_name visual_hull_4 \
    --white_background --render_path \
    --load_ply output/gaussian_object/kitchen/save/last.ply

Looking for config file in output/gs_init/kitchen/cfg_args
Config file found: output/gs_init/kitchen/cfg_args
Rendering output/gs_init/kitchen
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/vgg.pth
Using resized images in /content/GaussianObject/data/mip360/kitchen/images_4... [0

The rendering results are saved in `output/gs_init/kitchen/test/ours_None` and `output/gs_init/kitchen/render/ours_None`.

In [ ]:
from IPython.display import Video

Video("/content/GaussianObject/output/gs_init/kitchen/render/ours_None/renders.mp4", embed=True)